In [19]:
import os
import torch.backends.cudnn as cudnn
import torch.utils.data
from torchvision import transforms
# from dataset.data_loader import GetLoader
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import os
from skimage import io, transform
from os import listdir
import cv2
import numpy as np
import pandas as pd


class MNIST(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """ Intialize the MNIST dataset """
        self.root_dir = root_dir
        self.landmarks_frame = pd.read_csv(csv_file)
        self.transform = transform
        self.files = sorted(listdir(root_dir))
    def __getitem__(self, index):
        """ Get a sample from the dataset """
        img_name = os.path.join(self.root_dir,self.landmarks_frame.iloc[index,0])
        image = io.imread(img_name)
        # image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        label = self.landmarks_frame['label'][index]
        label = torch.FloatTensor([label])

        if self.transform:
            image = self.transform(image)
        return image, label

    def __len__(self):
        """ Total number of samples in the dataset """
        return len(self.landmarks_frame)

dataset_name = "SVHN"
epoch = 6
assert dataset_name in ['MNIST', 'SVHN']

model_root = os.path.join('.','model')
image_root = os.path.join('..', 'dataset', dataset_name)

cuda = True
cudnn.benchmark = True
batch_size = 128
image_size = 28
alpha = 0

"""load data"""

# img_transform_source = transforms.Compose([
#     transforms.Resize(image_size),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0.1307,), std=(0.3081,))
# ])

# img_transform_target = transforms.Compose([
#     transforms.Resize(image_size),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
# ])

if dataset_name == 'SVHN':
    # test_list = os.path.join(image_root, 'mnist_m_test_labels.txt')

    # dataset = GetLoader(
    #     data_root=os.path.join(image_root, 'mnist_m_test'),
    #     data_list=test_list,
    #     transform=img_transform_target
    # )
    dataset =  MNIST(csv_file="../../hw3_data/digits/svhn/test.csv", root_dir="../../hw3_data/digits/svhn/test",transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                       ]))
else:
    dataset = MNIST(csv_file="../../hw3_data/digits/mnistm/test.csv", root_dir="../../hw3_data/digits/mnistm/test",transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                       ]))

dataloader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=8
)

""" training """

my_net = torch.load(os.path.join(
    model_root, 'mnist_svhn_model' + str(epoch) + '.pth'
))
my_net = my_net.eval()

if cuda:
    my_net = my_net.cuda()

len_dataloader = len(dataloader)
data_target_iter = iter(dataloader)

i = 0
n_total = 0
n_correct = 0
features = []
labels = []
while i < len_dataloader:

    # test model using target data
    data_target = data_target_iter.next()
    t_img, t_label = data_target

    batch_size = len(t_label)

    input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
    class_label = torch.LongTensor(batch_size)

    if cuda:
        t_img = t_img.cuda()
        t_label = t_label.cuda()
        input_img = input_img.cuda()
        class_label = class_label.cuda()

    input_img.resize_as_(t_img).copy_(t_img)
    class_label.resize_as_(t_label.long()).copy_(t_label.long())

    class_output, _,_ = my_net(input_data=input_img, alpha=alpha)
#     print(feature.size())
#     print(feature.cpu())
#     features.append(feature.detach().cpu().numpy())
#     labels.append(t_label.detach().cpu().numpy())
#         pred = class_output.data.max(1, keepdim=True)[1]
#         n_correct += pred.eq(class_label.data.view_as(pred)).cpu().sum()
#         n_total += batch_size
#     print(i)
    i += 1

#     accu = n_correct.data.numpy() * 1.0 / n_total

#     print ('epoch: %d, accuracy of the %s dataset: %f' % (epoch, dataset_name, accu))

/home/robot/anaconda3/envs/env/lib/python3.7/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [21]:
len(t_img)

48

In [14]:
dataset.files

['00000.png',
 '00001.png',
 '00002.png',
 '00003.png',
 '00004.png',
 '00005.png',
 '00006.png',
 '00007.png',
 '00008.png',
 '00009.png',
 '00010.png',
 '00011.png',
 '00012.png',
 '00013.png',
 '00014.png',
 '00015.png',
 '00016.png',
 '00017.png',
 '00018.png',
 '00019.png',
 '00020.png',
 '00021.png',
 '00022.png',
 '00023.png',
 '00024.png',
 '00025.png',
 '00026.png',
 '00027.png',
 '00028.png',
 '00029.png',
 '00030.png',
 '00031.png',
 '00032.png',
 '00033.png',
 '00034.png',
 '00035.png',
 '00036.png',
 '00037.png',
 '00038.png',
 '00039.png',
 '00040.png',
 '00041.png',
 '00042.png',
 '00043.png',
 '00044.png',
 '00045.png',
 '00046.png',
 '00047.png',
 '00048.png',
 '00049.png',
 '00050.png',
 '00051.png',
 '00052.png',
 '00053.png',
 '00054.png',
 '00055.png',
 '00056.png',
 '00057.png',
 '00058.png',
 '00059.png',
 '00060.png',
 '00061.png',
 '00062.png',
 '00063.png',
 '00064.png',
 '00065.png',
 '00066.png',
 '00067.png',
 '00068.png',
 '00069.png',
 '00070.png',
 '0007